<a href="https://colab.research.google.com/github/gugi200/final_project/blob/main/scikit_models_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qU
!wandb login 3014974e724f01c4d63f956fa13fd7f0463e16d4
!pip install torchmetrics

In [5]:
# processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML libraries
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# torch functionality functions
import torch

# for downloading the data
from pathlib import Path
from torchvision.datasets import FashionMNIST
from torchvision import transforms

# helper functions
import wandb
from timeit import default_timer as timer
from tqdm.auto import tqdm